## ANALISIS DE DATOS DE FILMAFFINITY

In [12]:
import pandas as pd
import sqlite3

# ruta del csv
ruta_csv = "./data/filmaffinity_dataset.csv"

# Cargar el csv en un dataframe
df = pd.read_csv(ruta_csv)

# Conexión SQLite
conexion = sqlite3.connect("filmaffinity.db")

# Guardar el DataFrame como tabla en SQLite
df.to_sql("filmaffinity_tb", conexion, if_exists="replace", index=False)

print("Base de datos SQLite creada y datos importados")

cursor = conexion.cursor()

Base de datos SQLite creada y datos importados


In [13]:
consulta = """
           SELECT *
           FROM filmaffinity_tb
           LIMIT 10;
           """
resultado = pd.read_sql_query(consulta, conexion)
resultado

,Unnamed: 0,Título,Año,País,Dirección,Reparto,Nota,Tipo filme,Género
0,0,'49-'17,1917,Estados Unidos,Ruth Ann Baldwin,"Joseph W. Girard, Leo Pierson, William Dyer, M...",--,Película,Western
1,1,"10,000 Years B.C. (C)",1916,Estados Unidos,Willis H. O'Brien,None,"5,1",Cortometraje,Comedia
2,2,1812,1912,Rusia,"Vasili Goncharov, Kai Hansen, Aleksandr Uralsky","Pavel Knorr, Vasili Goncharov, Aleksandra Gonc...",--,Película,Drama
3,3,20.000 leguas de viaje submarino (C),1907,Francia,Georges Méliès,Georges Méliès,"6,0",Cortometraje,Fantástico
4,4,A Bad Case (C),1909,Francia,Émile Cohl,None,"5,3",Cortometraje,Comedia
5,5,A Bath Tub Elopement (C),1916,Estados Unidos,Marcel Perez,Marcel Perez,--,Cortometraje,Comedia
6,6,A Beast at Bay (C),1912,Estados Unidos,D.W. Griffith,"Mary Pickford, Edwin August, Alfred Paget, Mae...","5,4",Cortometraje,Drama
7,7,A Busy Night (C),1916,Estados Unidos,Marcel Perez,"Marcel Perez, Nilde Baracchi, Tom Murray",--,Cortometraje,Comedia
8,8,A Calamitous Elopement (C),1908,Estados Unidos,D.W. Griffith,"Harry Solter, Linda Arvidson, Charles Inslee, ...","4,6",Cortometraje,Comedia
9,9,A Child of the Ghetto (C),1910,Estados Unidos,D.W. Griffith,"Dorothy West, Kate Bruce, Dell Henderson, Char...",--,Cortometraje,Drama


# 1. Limpieza de data
En esta face haremos lo siguiente:

 1. Hacer un copia de la base de datos original para luego poder trabajar en esta copia y no afectar a la original.
 2. Luego pasamos a hacer la limpieza que consiste en:
      
      ✅ Datos faltantes (Nulos).

      ✅ Registros duplicados.

      ✅ Formatos inconsistentes (fechas, nombres, números).
      
      ✅ Valores atípicos (outliers).

### Hacer una copia de la db original para trabajar con esta copia

In [14]:
# Nombre de la nueva base de datos
copia_db = "./data/copia.db"

# Ejecutar SQL para hacer la copia
consulta = f"""
ATTACH DATABASE '{copia_db}' AS copia;
SELECT sql FROM sqlite_master WHERE type='table';
"""
cursor.executescript(consulta)

# Copiar cada tabla a la base de datos de respaldo
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()

for tabla in tablas:
    nombre_tabla = tabla[0]
    consulta_copia = f"CREATE TABLE copia.{nombre_tabla} AS SELECT * FROM {nombre_tabla};"
    cursor.execute(consulta_copia)

print("Copia de seguridad creada con SQL.")

Copia de seguridad creada con SQL.


In [16]:
consulta = "SELECT * FROM copia.filmaffinity_tb LIMIT 5;"
df_copia = pd.read_sql_query(consulta, conexion)
df_copia

,Unnamed: 0,Título,Año,País,Dirección,Reparto,Nota,Tipo filme,Género
0,0,'49-'17,1917,Estados Unidos,Ruth Ann Baldwin,"Joseph W. Girard, Leo Pierson, William Dyer, M...",--,Película,Western
1,1,"10,000 Years B.C. (C)",1916,Estados Unidos,Willis H. O'Brien,None,"5,1",Cortometraje,Comedia
2,2,1812,1912,Rusia,"Vasili Goncharov, Kai Hansen, Aleksandr Uralsky","Pavel Knorr, Vasili Goncharov, Aleksandra Gonc...",--,Película,Drama
3,3,20.000 leguas de viaje submarino (C),1907,Francia,Georges Méliès,Georges Méliès,"6,0",Cortometraje,Fantástico
4,4,A Bad Case (C),1909,Francia,Émile Cohl,None,"5,3",Cortometraje,Comedia


In [17]:
consulta_renombrar = "ALTER TABLE copia.filmaffinity_tb RENAME TO filmaffinity_copia;"
conexion.execute(consulta_renombrar)
conexion.commit()  # Guardar los cambios
print("Tabla renombrada correctamente.")

Tabla renombrada correctamente.


### ✅ Revisar datos faltantes (Nulos)

In [21]:
revisar_nulos = """
                SELECT COUNT(*) AS nulos -- Para que no ingore los NULL
                FROM copia.filmaffinity_copia
                WHERE Nota LIKE "%--%" OR Nota IS NULL OR TRIM(Nota) = '';
                """
resultado = pd.read_sql_query(revisar_nulos, conexion)
resultado

# Reemplazar las inconsistencias de notas por 0
#reemplazar = """
#            SELECT REPLACE(Nota, "--", "Sin_Calificar") AS Nota_Actualizada FROM copia.filmaffinity_copia;
#            """
#resultado_2 = pd.read_sql_query(reemplazar, conexion)
#resultado_2

,nulos
0,50709


In [22]:
info_tabla = "PRAGMA table_info(filmaffinity_copia);"
info_tabla = pd.read_sql_query(info_tabla, conexion)
info_tabla

,cid,name,type,notnull,dflt_value,pk
0,0,Unnamed: 0,INT,0,None,0
1,1,Título,TEXT,0,None,0
2,2,Año,INT,0,None,0
3,3,País,TEXT,0,None,0
4,4,Dirección,TEXT,0,None,0
5,5,Reparto,TEXT,0,None,0
6,6,Nota,TEXT,0,None,0
7,7,Tipo filme,TEXT,0,None,0
8,8,Género,TEXT,0,None,0


In [23]:
revisar_nulos = """
                SELECT COUNT(*) AS nulos -- Para que no ingore los NULL
                FROM copia.filmaffinity_copia
                WHERE Nota LIKE "%--%" OR Nota IS NULL OR TRIM(Nota) = '';
                """
resultado = pd.read_sql_query(revisar_nulos, conexion)
resultado

,nulos
0,50709


In [25]:
actualizar = """
            UPDATE copia.filmaffinity_copia
            SET Nota = "Sin Calificar"
            WHERE Nota = "--"
            """
conexion.execute(actualizar)
conexion.commit()
print("Notas actualizadas correctamente")

Notas actualizadas correctamente


In [26]:
actualizar_n = """
               UPDATE copia.filmaffinity_copia
               SET Nota = REPLACE(Nota, ",", ".")
               WHERE Nota LIKE "%,%";
               """
conexion.execute(actualizar_n)
conexion.commit()
print("Valores Actualizados correctamente")

Valores Actualizados correctamente


In [ ]:
# Valores Nulos en la columna Reparto
reparto_nulos = """
                SELECT COUNT(*) as total_nulos
                FROM copia.filmaffinity_copia
                WHERE Reparto IS NULL;
               """
reparto_nulos = pd.read_sql_query(reparto_nulos, conexion)
reparto_nulos # Y lo dejamos como NULL porque no afecta al análisis

,total_nulos
0,24478


In [ ]:
# Cerrar conexión con
# conexion.close()